# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [77]:
 import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [78]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [79]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
             
        for line in csvreader:
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [80]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [81]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [82]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS song 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [83]:
try:
    session.set_keyspace('song')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Quesion 1:

#### The Question 1 expects Name of the artist, title of the song and length of the track based on sessionId and itemInSession. Primary Key: The PRIMARY key for the table should uniquely identify each row in the table. For us we need results based on sessionId and itemInSession; so we neeed these both as the primary key. After inserting the table with correct data type, the result is shown with SELECT and WHERE clause.

In [84]:
query = "CREATE TABLE IF NOT EXISTS song_length "
query = query + "(sessionId int, itemIdSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemIdSession))" 
try:
    session.execute(query)
except Exception as e:
    print(e)

In [85]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_length(sessionId, itemIdSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), str(line[0]), str(line[9]), float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

## Quesion 1 Answer

In [86]:
query = "SELECT artist, song, length FROM song_length WHERE sessionId = 338 AND itemIdSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


## Qestion 2

#### The Question 2 expects Name of the artist, title of the song and user based on certain userId and sessionId. Primary Key: The PRIMARY key for the table should uniquely identify each row in the table. For us we need results based on userId and sessionId; so we neeed these both as the primary key, the table is sorted based on itemInSession. After inserting the table with correct data type, the result is shown with SELECT and WHERE clause.

In [87]:
query = "CREATE TABLE IF NOT EXISTS song_user_sess"
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, \
PRIMARY KEY ((userId, sessionId), itemInSession))" 
try:
    session.execute(query)
except Exception as e:
    print(e)

In [88]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_user_sess(userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), str(line[0]), str(line[9]), str(line[1]), str(line[4])))

## Question 2 Answer:

In [89]:
query = "SELECT artist, song, firstname, lastname FROM song_user_sess WHERE userID = 10 AND sessionid = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


## Question 3:

#### The Question 3 expects Name of the username based on certain song. Primary Key: The PRIMARY key for the table should uniquely identify each row in the table. For us we need results based on song; so we neeed these song as the primary key, the table is sorted based on userId. After inserting the table with correct data type, the result is shown with SELECT and WHERE clause.

In [90]:
query = "CREATE TABLE IF NOT EXISTS user"
query = query + "(song text, userId int, firstName text, lastName text, \
PRIMARY KEY (song, userId))" 
try:
    session.execute(query)
except Exception as e:
    print(e)

In [91]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user(song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (str(line[9]), int(line[10]), str(line[1]), str(line[4])))

## Question 3 Answer:

In [92]:
query3 = "SELECT firstName, lastName FROM user WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [93]:
query = "DROP TABLE IF EXISTS song_length"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [94]:
query = "DROP TABLE IF EXISTS song_user_sess"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    

In [95]:
query = "DROP TABLE IF EXISTS user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [96]:
session.shutdown()
cluster.shutdown()